<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/TrueCase_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **This is an in-progress draft!**

In [ ]:
! pip install -Uq transformers sentence-transformers datasets

In [26]:
from datasets import load_dataset
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import random
from tqdm.auto import tqdm
from typing import List
import re
from transformers import AutoTokenizer
import math
random.seed(1234)
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# dataset = load_dataset("wikitext", "wikitext-103-v1", split="train", streaming=True)
dataset = load_dataset("cnn_dailymail", '3.0.0', split="train", streaming=True)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [5]:
def num_word_tokens(text: str) -> List[str]:
  word_tokens = word_tokenize(text)
  return len(word_tokens)

def num_sent_tokens(text: str) -> List[str]:
  sentence_tokens = sent_tokenize(text)
  return len(sentence_tokens)

assert num_word_tokens("This is great") == 3
assert num_sent_tokens("This is great. This is sentence 2.") == 2

In [29]:
def create_dataset(limit: int) -> List[str]:
  sentences = []
  progress_bar_total = tqdm(range(dataset["train"].dataset_size), desc="Articles")
  progress_bar_sentences = tqdm(range(limit), desc="Sentences")
  for idx, dict_ in enumerate(dataset["train"]):
    progress_bar_total.update(1)
    if len(sentences) > limit:
      break
    text = dict_["article"]
    if word_tokenize(text) and len(sent_tokenize(text)) > 4:
      sentences_extracted = sent_tokenize(text)[3:]
      # sentences_extracted = list(map(preprocess_text, sentences_extracted))
      sentences_extracted = list(filter(lambda x: num_word_tokens(x) > 5, sentences_extracted))
      if sentences_extracted:
        sentences.extend(sentences_extracted)
        progress_bar_sentences.update(len(sentences_extracted))
  
  sentences = random.sample(sentences, limit)
  split = math.ceil(0.8 * len(sentences))
  train_sentences, validation_sentences = sentences[:split], sentences[split:]
  return train_sentences, validation_sentences

In [32]:
train_sentences, validation_sentences = create_dataset(limit=100000)
print(f"Total train sentences: {len(train_sentences)}")
print(f"Total validation sentences: {len(validation_sentences)}")

Articles:   0%|          | 0/1369362325 [00:00<?, ?it/s]

Sentences:   0%|          | 0/100000 [00:00<?, ?it/s]

80000
Total train sentences: 80000
Total validation sentences: 20000


In [33]:
# Check some train sentences
random.sample(train_sentences, 10)

['He has called for an immediate cease-fire between Hamas and Israel and said he is encouraged that the Egyptian government is trying to broker a truce.',
 'In a recent video that has made the rounds on the Internet, a student poses tough questions to the president of Cuba\'s National Assembly, asking why Cubans cannot travel freely to resorts -- a practice derided by critics as "tourist apartheid."',
 'This is particularly vital now given the recent election results.',
 'The reason for his rejection was unclear.',
 'The prince, who is called sub lieutenant Wales in the navy, is also expected to spend time aboard a mine hunter and submarine during his attachment, which ends August 1.',
 'In those days, he had difficulty concentrating, for which he was beaten and taunted at school.',
 'America has gone more than seven years without another terrorist attack on our soil."',
 "Meanwhile, the Library of Congress marks the bicentennial Thursday by opening a special exhibit featuring Lincoln'

In [34]:
# Check some validation sentences
random.sample(validation_sentences, 10)

['The tanker was carrying two million barrels of crude oil worth about $100 million.',
 '"It\'s pretty clear that the administration just wants to muzzle any comments that reflect negatively on the [al-]Maliki government," he said.',
 'Legend has it that when grandpa arrived from Ireland, Germany, Italy or Poland, he jumped off the boat, immediately draped himself in the American flag, ripped out his native tongue, and abandoned his culture -- all while singing "Yankee Doodle Dandy."',
 '"No casualty is more or less significant than another; each soldier, Marine, airman and sailor is equally precious and their loss equally tragic," said Rear Adm. Gregory Smith, the U.S. military\'s chief spokesman in Iraq.',
 'It\'s all that\'s right about wine."',
 '"It\'s about our fundamental values," he said.',
 'There were 18 inmates in the jail at the time of the tornado, and they were moved to neighboring jurisdictions, he said.',
 'Watch Larry King interview Susan Boyle » .',
 'A quart bottle o